In [ ]:
# -*- coding: utf-8 -*-
import os, warnings
warnings.filterwarnings("ignore", category=UserWarning)
os.environ["OMP_NUM_THREADS"]="1"; os.environ["OPENBLAS_NUM_THREADS"]="1"; os.environ["MKL_NUM_THREADS"]="1"

import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GroupShuffleSplit, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from scipy.stats import spearmanr, uniform, randint
from pathlib import Path
from sklearn.isotonic import IsotonicRegression
import matplotlib.pyplot as plt # [추가] 산점도를 위한 라이브러리

# =========================================
# 경로 / 상수
# =========================================
EDGE_FP  = 'data/마포_서대문_은평_도로망_최종_v2.csv'
LABEL_FP = 'data/은마서_교통사고.csv'
OUT_DIR  = 'data/xgb_outputs_calibrated'
EDGE_OUT = str(Path(OUT_DIR, '은마서_도로_위험도_calibrated.csv'))
METRIC_OUT = str(Path(OUT_DIR, "test_metrics_calibrated.csv"))
PRED_OUT   = str(Path(OUT_DIR, "test_predictions_calibrated.csv"))
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

RANDOM_STATE = 123
N_ITER = 200

# =========================================
# 유틸 함수 (기존과 동일)
# =========================================
def to_pair(a, b):
    a, b = int(a), int(b)
    return (a, b) if a <= b else (b, a)

def make_tail_weights(y, lam=5.0, p=2.0):
    y_norm = y / max(y.max(), 1e-9)
    weights = 1.0 + lam * np.power(y_norm, p)
    return weights

# =========================================
# 1) 데이터 로드 및 병합 (기존과 동일)
# =========================================
print("▶ 데이터 로딩 및 병합...")
edges = pd.read_csv(EDGE_FP, dtype={'u': str, 'v': str})
edges['u'] = pd.to_numeric(edges['u']); edges['v'] = pd.to_numeric(edges['v'])
labels = pd.read_csv(LABEL_FP)
edges['pair'] = list(map(to_pair, edges['u'], edges['v']))
labels['pair'] = list(map(to_pair, labels['node1'], labels['node2']))
data = edges.merge(labels[['pair', 'ratio']], on='pair', how='left')
data['ratio'] = data['ratio'].fillna(0)
data = data.rename(columns={'ratio': 'y'})
data = data.groupby('pair').agg({
    **{col: 'first' for col in edges.columns if col != 'pair'}, 'y': 'mean'
}).reset_index()

# =========================================
# 2) 데이터 분할 (8:1:1) (기존과 동일)
# =========================================
print("\n▶ 데이터 분할 (80% train, 10% validation, 10% test)...")
cat_cols = [c for c in ['highway','surface'] if c in data.columns]
num_cols = [c for c in ['oneway','bridge','tunnel','length','lanes','width'] if c in data.columns]
feature_cols = cat_cols + num_cols
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_STATE)
train_idx, holdout_idx = next(gss1.split(data, groups=data['pair']))
train_df = data.iloc[train_idx]
holdout_df = data.iloc[holdout_idx]
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.5, random_state=RANDOM_STATE)
val_idx, test_idx = next(gss2.split(holdout_df, groups=holdout_df['pair']))
val_df = holdout_df.iloc[val_idx]
test_df = holdout_df.iloc[test_idx]
print(f"  train={len(train_df)}  validation={len(val_df)}  test={len(test_df)}")

# =========================================
# 3) 전처리 및 가중치 생성 (기존과 동일)
# =========================================
print("\n▶ 데이터 전처리 및 가중치 생성...")
pre = ColumnTransformer([
    ('cat', Pipeline([('imp', SimpleImputer(strategy='constant')), ('ohe', OneHotEncoder(handle_unknown='ignore'))]), cat_cols),
    ('num', Pipeline([('imp', SimpleImputer(strategy='median')), ('scaler', StandardScaler())]), num_cols)
], remainder='drop')
X_tr = pre.fit_transform(train_df[feature_cols])
X_va = pre.transform(val_df[feature_cols])
X_te = pre.transform(test_df[feature_cols])
y_tr = train_df['y'].values
y_va = val_df['y'].values
y_te = test_df['y'].values
sample_weights = make_tail_weights(y_tr)

# =========================================
# 4) RandomizedSearchCV를 이용한 최적화 (기존과 동일)
# =========================================
print(f"\n▶ RandomizedSearchCV 최적화 시작 (R2 점수 기준, 탐색 횟수: {N_ITER})...")
param_dist = {
    'learning_rate': uniform(0.01, 0.04), 'n_estimators': randint(1500, 4000),
    'max_depth': randint(6, 12), 'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.6, 0.4), 'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(1, 20), 'min_child_weight': randint(1, 10)
}
model = xgb.XGBRegressor(
    objective='reg:squarederror', eval_metric='rmse',
    early_stopping_rounds=80, random_state=RANDOM_STATE,
    tree_method='hist', device='cuda'
)
random_search = RandomizedSearchCV(
    model, param_distributions=param_dist, n_iter=N_ITER, scoring='r2',
    cv=3, verbose=2, random_state=RANDOM_STATE, n_jobs=-1
)
fit_params = { 'sample_weight': sample_weights, 'eval_set': [(X_va, y_va)], 'verbose': False }
random_search.fit(X_tr, y_tr, **fit_params)
print("\n최적화 종료!")
print("최적의 파라미터:"); print(random_search.best_params_)
best_model = random_search.best_estimator_

# =========================================
# 5) 캘리브레이션 모델 학습
# =========================================
print("\n▶ 캘리브레이션 모델 학습 (Validation Set)...")
val_pred_raw = best_model.predict(X_va)
calibrator = IsotonicRegression(out_of_bounds="clip")
calibrator.fit(val_pred_raw, y_va)
print("캘리브레이터 학습 완료.")

# =========================================
# 6) 최종 모델 평가 (Test Set)
# =========================================
print("\n▶ 최종 평가 (Test Set)...")
y_pred_raw = best_model.predict(X_te)
y_pred_calibrated = calibrator.predict(y_pred_raw)

r2 = r2_score(y_te, y_pred_calibrated)
rmse = np.sqrt(mean_squared_error(y_te, y_pred_calibrated))
mae = mean_absolute_error(y_te, y_pred_calibrated)
spearman = spearmanr(y_te, y_pred_calibrated).correlation
metrics = pd.Series({"R2": r2, "RMSE": rmse, "MAE": mae, "Spearman": spearman})
print("Hold-out Test Set 최종 평가 결과 (캘리브레이션 적용):")
print(metrics.round(6))

metrics.to_csv(METRIC_OUT)
pd.DataFrame({'y_true': y_te, 'y_pred_raw': y_pred_raw, 'y_pred_calibrated': y_pred_calibrated}).to_csv(PRED_OUT, index=False)
print(f"\n[SAVED] Test 평가 지표 -> {METRIC_OUT}")


# =========================================
# [추가] 7) 산점도 시각화
# =========================================
print("\n▶ 산점도 시각화...")
plt.figure(figsize=(10, 10))

# 보정 전 예측값 (회색, 반투명)
plt.scatter(y_te, y_pred_raw, alpha=0.3, label='Before Calibration (보정 전)', color='grey', s=15)

# 보정 후 예측값 (파란색)
plt.scatter(y_te, y_pred_calibrated, alpha=0.5, label='After Calibration (보정 후)', color='blue', s=20)

# 완벽한 예측선 (y=x)
line_max = max(y_te.max(), y_pred_calibrated.max()) * 1.05
plt.plot([0, line_max], [0, line_max], 'k--', label='Perfect Prediction (y=x)')

plt.xlabel("Actual Risk (y_true)")
plt.ylabel("Predicted Risk (y_pred)")
plt.title("Prediction vs. Actual Risk (Before & After Calibration)")
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.xlim(0, line_max)
plt.ylim(0, line_max)
plt.gca().set_aspect('equal', adjustable='box')

# 그래프 저장
plot_path = os.path.join(OUT_DIR, "scatterplot_calibrated.png")
plt.savefig(plot_path, dpi=200)
print(f"[SAVED] 산점도 -> {plot_path}")
plt.show()


# =========================================
# 8) 전체 도로망 스코어링
# =========================================
print("\n▶ 전체 도로망 위험도 스코어링...")
X_scoring = pre.transform(data[feature_cols])
scoring_pred_raw = best_model.predict(X_scoring)
scoring_pred_calibrated = calibrator.predict(scoring_pred_raw)

data['risk_score'] = scoring_pred_calibrated
output_cols = ['u', 'v', 'osmid', 'highway', 'length', 'lanes', 'risk_score']
output_cols_exist = [c for c in output_cols if c in data.columns]
data[output_cols_exist].to_csv(EDGE_OUT, index=False)
print(f"[SAVED] 전체 도로 위험도 -> {EDGE_OUT}")

# (이하 파라미터 저장 부분은 기존과 동일)